In [2]:
%matplotlib notebook

In [32]:
import time
import numpy as np
import pandas as pd
import xarray as xr

from collections import OrderedDict
from importlib import reload

In [4]:
from plottr import plottr
reload(plottr)

<module 'plottr.plottr' from 'd:\\Dropbox\\Code\\labpyDesktop\\plottr\\plottr\\plottr.py'>

In [5]:
x = np.linspace(0, 8, 21, dtype=float)
# y = np.arange(9, dtype=float)
# xx, yy = np.meshgrid(x, y, indexing='ij')
# zz = xx + yy**2
z = np.cos(x)

# x2 = xx.reshape(-1)[:]
# y2 = yy.reshape(-1)[:]
# z2 = zz.reshape(-1)[:]
x2 = x
z2 = z

data = {
    'id' : 'Some data',
    'datasets' : {
        'z' : {
            'values' : z2.tolist(),
            'axes' : ['x', ],
        },
        'x' : {
            'values' : x2.tolist(),
        },
#         'y' : {
#             'values' : y2,
#         },
#         'w' : {
#             'values' : z2,
#         }
    }
}

In [6]:
dfs = plottr.dictToDataFrames(data['datasets'])

In [22]:
arr = plottr.dataFrameToXArray(dfs[0])

In [31]:
arr.coords['x'].values

array([0. , 0.4, 0.8, 1.2, 1.6, 2. , 2.4, 2.8, 3.2, 3.6, 4. , 4.4, 4.8,
       5.2, 5.6, 6. , 6.4, 6.8, 7.2, 7.6, 8. ])

In [41]:
vals = OrderedDict({'x' : {'nVals' : 3}, 'y' : {'nVals' : 4}, })

In [44]:
[n for n, k in vals.items()]

['x', 'y']